<a href="https://colab.research.google.com/github/NehaKumari500092077/Machine-Learning-Lab/blob/main/Session_2_Assignments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Task - 1:
Dataset creation using make_regression of scikit-learn [Mark - 0]

In [ ]:
# Creating regression dataset
# from sklearn.datasets import make_regression
# import pandas as pd

# # Creating a toy regression dataset with 5 features
# X_gen, y_gen = make_regression(n_samples=1000, n_features=5, noise=0.1, random_state=42)

# # Combine the features and target into one DataFrame
# df = pd.DataFrame(X_gen, columns=[f'Feature_{i}' for i in range(1, 6)])
# df['target'] = y_gen

# # Save the DataFrame to a CSV file
# df.to_csv('regression_dataset.csv', index=False)

In [1]:
import numpy as np
import pandas as pd
# Importing the dataset
dataset = pd.read_csv('https://raw.githubusercontent.com/Rekha215/Machine-Learning-Laboratory/main/regression_dataset.csv')

In [2]:
# We can get a quick idea of how many instances (rows) and how many attributes (columns) the data contains with the shape property.
dataset.shape

(1000, 6)

In [3]:
dataset.head(10)

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,target
0,2.056544,0.606851,0.482688,-1.130888,0.420094,74.905613
1,-0.799192,-0.645964,-0.182896,-0.482744,1.374876,-41.649588
2,1.076007,-0.796026,-0.751969,0.021312,-0.319054,-24.182866
3,-0.103255,-0.828497,1.489863,-1.643189,-1.600904,-87.094505
4,-2.063403,0.503252,-0.645572,-0.317350,-1.661083,-86.122029
5,-0.215610,-0.007045,-0.451949,-0.371858,-0.789339,-38.209953
6,0.613518,0.120031,0.436324,-1.022793,0.038003,5.926908
7,0.234024,0.553040,1.818062,-0.248529,0.775155,71.377873
8,1.588627,0.140983,0.145521,0.695402,-0.614123,59.912602
9,0.182501,-0.044893,-0.399899,-0.670115,0.269295,-15.096912


#### Task - 2:
Two K-NN regressor implementations (from scratch)
1. Average of K-nearest neighbors [Marks - 5]
2. Weighted average of k nearest neighbors, with weights being inverse of distances from the k neighbors [Marks - 5]

#### Task - 3:
Plot regression metric (mean squared error) against different values of k ``` {3, 7, 11}``` and different distance measures ```{p=2, p=1, p=5}```and report the k value and distance measure combination that gave the best performance (lowest MSE) [Marks - 4]

#### Task - 4:
Implement KNN regression using scikit-learn, with the same values of k and p as above

##### Note - one argument of scikit-learn KNN is ```algorithm='auto'``` and acceptable values are ```{'auto', 'ball_tree', 'kd_tree', 'brute'}, default='auto' ```. For this task use ```'brute'``` setting.

[Marks - 4]

#### Task - 5:
Comparison with scikit-learn's implementation - Does the optimal values of k and distance metric as well as best performance level of the from scratch implementation match with those given by Sklearn?

 [Marks - 2]